# 1. if you know Numpy (page5)

In [1]:
import numpy as np
a = np.zeros((2,2));
b = np.ones((2,2));
np.sum(b, axis=1)
a.shape

(2, 2)

In [2]:
np.reshape(a, (1,4))

array([[ 0.,  0.,  0.,  0.]])

# 2. Repeat in TensorFlow (page6)

In [3]:
import tensorflow as tf
sess = tf.Session()
a = tf.zeros((2,2));
b = tf.ones((2,2));
sess.run(tf.reduce_sum(b, axis=1))

array([ 2.,  2.], dtype=float32)

In [4]:
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [5]:
sess.run(tf.reshape(a, (1,4)))

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

In [6]:
sess = tf.Session()
a = np.zeros((2,2));
ta = tf.zeros((2,2));
print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [7]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


# 3. Computation Graph (page9)

In [8]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

In [9]:
sess = tf.Session()
print(sess.run(c))

30.0


In [10]:
with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())

30.0
30.0


# 4. Tensorflow Variables (page14)

In [11]:
w = tf.Variable(tf.random_normal([5,2], stddev=0.1), name="weight")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[ 0.26146415  0.11835181]
 [-0.03477342 -0.05319005]
 [-0.00053301  0.15901326]
 [ 0.06963708 -0.00223109]
 [ 0.11484361  0.16059124]]


In [12]:
state = tf.Variable(0, name="counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


# 5. Tensorflow Placeholder (page26)

In [13]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul = tf.multiply(x1, temp) #정확한 연산자는 tf.multiply
with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


In [14]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b)

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a: 2, b: 3}))
    print(sess.run(mul, feed_dict={a: 2, b: 3}))

5
6


In [15]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    result = sess.run(product)
    print(result)

[[ 12.]]


In [16]:
import numpy as np
matrix1 = tf.placeholder(tf.float32, [1, 2])
matrix2 = tf.placeholder(tf.float32, [2, 1])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], [2.]])
    result = sess.run(product, feed_dict={matrix1: mv1, matrix2: mv2})
    print(result)

[[ 12.]]


# 6. MNIST with MLP (page27)

In [21]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

learning_rate = 0.001
max_steps = 15000
batch_size = 128

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

def MLP(inputs):
    W_1 = tf.Variable(tf.random_normal([784, 256]))
    b_1 = tf.Variable(tf.zeros([256]))
    
    W_2 = tf.Variable(tf.random_normal([256, 256]))
    b_2 = tf.Variable(tf.zeros([256]))
    
    W_out = tf.Variable(tf.random_normal([256, 10]))
    b_out = tf.Variable(tf.zeros([10]))
    
    h_1 = tf.add(tf.matmul(inputs, W_1), b_1)
    h_1 = tf.nn.relu(h_1)
    
    h_2 = tf.add(tf.matmul(h_1, W_2), b_2)
    h_2 = tf.nn.relu(h_2)
    
    out = tf.add(tf.matmul(h_2, W_out), b_out)
    
    return out

net = MLP(x)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y))
#logits과 labels를 설정해주어야함
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)

init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)

for step in range(max_steps):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, loss = sess.run([opt, loss_op],
                      feed_dict={x: batch_x, y:batch_y})
    if(step+1) % 1000 == 0:
        print("[{}/{}] loss{:.3f}".format(step+1, max_steps, loss))
print("Optimization Finished!")

correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Train accuray: {:.3f}".format(sess.run(accuracy,
        feed_dict={x: mnist.train.images, y: mnist.train.labels})))
print("Test accuray: {:.3f}".format(sess.run(accuracy,
        feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[1000/15000] loss38.371
[2000/15000] loss7.422
[3000/15000] loss5.568
[4000/15000] loss7.523
[5000/15000] loss5.615
[6000/15000] loss0.519
[7000/15000] loss0.088
[8000/15000] loss0.993
[9000/15000] loss0.000
[10000/15000] loss0.095
[11000/15000] loss0.000
[12000/15000] loss0.038
[13000/15000] loss0.000
[14000/15000] loss1.348
[15000/15000] loss0.000
Optimization Finished!
Train accuray: 0.996
Test accuray: 0.958


# 7. Variable Scope (page33)

In [22]:
var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        var2 = tf.Variable([1], name="var")
        var3 = tf.Variable([1], name="var")
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var:0
var2: foo/bar/var:0
var3: foo/bar/var_1:0


In [23]:
var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var2 = tf.Variable([1], name="var")
        scp.reuse_variables()
        var3 = tf.Variable([1], name="var")
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var_1:0
var2: foo_1/bar/var:0
var3: foo_1/bar/var_1:0


In [26]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var1 = tf.get_variable("var", [1])
        scp.reuse_variables()
        var2 = tf.get_variable("var", [1])
    
    with tf.variable_scope("bar", reuse=True):
        var3 = tf.get_variable("var", [1])
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: foo/bar/var_2:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


# 8. Wrappers (page44)

In [27]:
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    bottom_shape = bottom.get_shape().as_list()[3]
    with tf.variable_scope(scope or "conv"):
        w = tf.get_variable("w", [ksize, ksize, bottom_shape, num_filter], initializer = he_init)
        b = tf.get_variable("b", [num_filter], initializer = tf.constant_initializer(0))
        
        x = tf.nn.conv2d(bottom, w, strides = [1, stride, stride, 1], padding = padding)
        x = tf.nn.relu(tf.nn.bias_add(x,b))
        
    return x
    

In [28]:
def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides = [1, stride, stride, 1], padding=padding)
    return pool

In [29]:
def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        w = tf.get_variable("w", [ksize, ksize, bottom_shape, num_filter], initializer = he_init)
        b = tf.get_variable("b", [num_filter], initializer = tf.constant_initializer(0))
        
        out = tf.nn.bias_add(tf.matmul(bottom, w), b)
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
    return relu

In [30]:
keep_prob = tf.placeholder(tf.float32, None)

def conv_net(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope="out")
    return out

In [31]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

In [37]:
def save(self, ckpt_dir, global_step=None):
    if self.config.get("saver") is None:
        self.config["saver"] = \
            tf.train.Saver(max_to_keep=30)
    
    saver = self.config["saver"]
    sess = self.config["sess"]
    
    dirname = os.path.join(ckpt_dir, self.name)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    saver.save(sess, dirname, global_step)

In [38]:
def load_latest_checkpoint(self, ckpt_dir, exclude=None):
    path = tf.train.latest_checkpoint(ckpt_dir)
    if path is None:
        raise AssertionError("No ckpt exists in {0}.".format(ckpt_dir))
        
    print("Load {} save file".format(path))
    self._load(path, exclude)
    
def load_from_path(self, ckpt_path, exclude=None):
    self._load(ckpt_path, exclude)
    
def _load(self, ckpt_path, exclude):
    init_fn = slim.assign_from_checkpoint_fn(ckpt_path, 
            slim.get_variables_to_restore(exclude=exclude),
            ignore_missing_vars=True)
    init_fn(self.config["sess"])

# 9. Vgg-16 (page78)

NameError: name 'slim' is not defined

In [40]:
x = tf.placeholder(tf.float32, [None, 224, 224, 3], name="x")
y = tf.placeholder(tf.int32, [None, 8], name="y")
is_training = tf.placeholder(tf.bool, name="is_training")
with slim.arg_scope(vgg.vgg_arg_scope()):
    net, end_pts = vgg.vgg_16(x, is_training=is_training, num_classes=1000)
    
with tf.variable_scope("losses"):
    cls_loss = slim,losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = type_loss + reg_loss
    
with tf.variable_scope("opt"):
    opt = tf.train.AdamOptimizer(0.001).minimize(loss_op)
    
self.load_from_path(ckpt_path=VGG_PATH, exclude=["vgg_16/fc8"])

NameError: name 'slim' is not defined

# 10.Tensorboard & slim example (page82)

In [45]:
import tensorflow as tf
slim = tf.contrib.slim

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

max_steps = 10000
batch_size = 128
lr = 0.001
keep_prob = 0.5
weight_decay = 0.0004
logs_path = "/tmp/tensorflow_logs/example"

def my_arg_scope(is_training, weight_decay):
    with slim.arg_scope([slim.conv2d], 
        activation_fn=tf.nn.relu,
        weights_regularizer=slim.l2_regularizer(weight_decay),
        weights_initializer=slim.variance_scaling_initializer(),
        biases_initializer=tf.zeros_initializer,
        stride=1, padding="SAME"):
        with slim.arg_scope([slim.dropout], is_training=is_training) as arg_sc:
            return arg_sc
        
def my_net(x, keep_prob, outputs_collections="my_net"):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    with slim.arg_scope([slim.conv2d, slim.max_pool2d], outputs_collections=outputs_collections):
        net = slim.conv2d(x, 64, [3,3], scope="conv1")
        net = slim.max_pool2d(net, [2,2], scope="pool1")
        net = slim.conv2d(net, 128, [3,3], scope="conv2")
        net = slim.max_pool2d(net, [2,2], scope="pool2")
        net = slim.conv2d(net, 256, [3,3], scope="conv3")
        net = tf.reduce_mean(net, [1, 2], name="pool3", keep_dims=True)
        net = slim.dropout(net, keep_prob, scope="dropout3")
        net = slim.conv2d(net, 1024, [1,1], scope="fc4")
        net = slim.dropout(net, keep_prob, scope="dropout4")
        net = slim.conv2d(net, 10, [1,1], activation_fn=None, scope="fc5")
        
    end_points = \
        slim.utils.convert_collection_to_dict(outputs_collections)
    return tf.reshape(net, [-1, 10]), end_points

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with slim.arg_scope(my_arg_scope(is_training, weight_decay)):
    net, end_pts = my_net(x, keep_prob)
    pred = slim.softmax(net, scope="prediction")
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = cls_loss + reg_loss
    
with tf.variable_scope("Adam"):
    opt = tf.train.AdamOptimizer(lr)
    grads = tf.gradients(loss_op, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    apply_grads = opt.apply_gradients(grads_and_vars=grads)
    
with tf.variable_scope("accuracy"):
    correct_op = tf.equal(tf.argmax(net, 1), tf.argmax(y,1))
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))

summ_loss = tf.summary.scalar("loss", loss_op)
summ_acc = tf.summary.scalar("accuracy_test", acc_op)

for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var, collections=["my_summ"])
for grad, var in grads:
    tf.summary.histogram(var.name + "/gradient", grad, collections=["my_summ"])

    summ_wg = tf.summary.merge_all(key="my_summ")
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

for step in range(max_steps):
    batch_x, bath_y = mnist.train.next_batch(batch_size)
    _, loss, plot_loss, plot_wg = sess.run([apply_grads, loos_op, summ_loss, summ_wg],
                                          feed_dict={x: batch_x, y: batch_y, is_training: True})
    summary_writer.add_summary(plot_loss, step)
    summary_writer.add_summary(plot_wg, step)
    
    if(step+1) % 100 == 0:
        ploat_acc = sess.run(summ_acc, feed_dict={x: mnist.test.images,
                                                 y: mnist.test.labels,
                                                 is_training: False})
        summary_writer.add_summary(plot_acc, step)
    print("Optimization Finished!")
    
    test_acc =sess.run(acc_op, feed_dict={x: mnist.test.images,
                                         y: mnist.test.labels,
                                         is_training: False})
    print("Test accuracy: {:.3f}".format(test_acc))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


TypeError: __init__() got an unexpected keyword argument 'partition_info'